# Overview
This notebook contains the implementation of the Multivariate Sea Storm Model (https://github.com/anardek/Multivariate_Sea_Storm_Model/tree/main) used to generate 20,000 synthetic storms parameterized for North Core Banks, NC using a 30-year historical time record. Below, I provide a summary of the MSSM components and use within Barrier3D. The storms developed in this current notebook are used within the manuscript "XXXXXXX" Van Blunk et al., (in prep).

# Storms in Barrier3D
Storms in Barrier3D are be described by 3 variables: storm duration, the highest runup elevation (Rhigh), and the lowest runup elevation (Rlow). In this notebook, we describe how to generate a list of synthetic storms for use in Barrier3D following the methods of Wahl et al. (2016) for a copula-based multivariate sea-storm model (MSSM). In this notebook, the MSSM model is developed using wave hindcast and water-level data from the North Core Banks, NC and accounts for interdependencies between these three variables. We then illustrate how the list of synthetic storms can be used to generate annual storm time series (i.e., many storms per model year) for use in Barrier3D. Most of the text below describing the MSSM is taken from the Supplementary Text of Reeves et al., 2021 and the notebook here: https://github.com/UNC-CECL/Barrier3D/blob/master/notebooks/MSSM_for_Barrier3d.ipynb).

## Multivariate Sea Storm Model
The MSSM model `multivariateSeaStorm.m` (located at https://github.com/anardek/Multivariate_Sea_Storm_Model) was written in Matlab in order to utilize the t-tide package, which allows for robust fitting of tidal constituents to water level time series, and therefore sadly cannot be run in this notebook. We provide a sample call below and describe the function components: 

`[stStorms, stSimStorms] = multivariateSeaStorm(sCopula, sWIS_filename, sWaterLevel_filename, fBeta, fBermEl, nSimStorm, bPlot, sOutput_filename)`

The MSSM model requires the following inputs: 

`
% Inputs:
%       sCopula              - copula to be fitted to storm variables:"c-vine", "d-vine", "gaussian"
%       sWIS_filename        - .onlns file downloaded from a USACE Wave Information Studies (WIS) bouy; must 
%                              contain hourly records of wave height (m)
%       sWaterLevel_filename - .txt file that contains hourly records of total water level in m NAVD88 as second 
%                              column, first column is datetime; downloaded for each year from NOAA;
%                              must be either the same length or longer than WIS time record
%       fBeta                - beach slope 
%       nSimStorm            - number of simulated storms to create
%       fBermEl              - erosion threshold; Wahl used 5% of dune toe heights; we use the average berm 
%                              elevation (m NAVD88)
%       bPlot                - boolean for plotting
%       sOutputFilename      - string of prefix for csv output filename
`

For North Core Banks, we utilize a 30 year record of hourly wave hindcast data – including wave height (Hs) and wave period (Tp) – from the USACE’s Wave Information Studies buoy offshore North Core Banks (ST63266, 24 m water depth) and hourly records of water level from the nearest NOAA tide gauge (Station 8656483, Beaufort, NC) to create a list of 20,000 synthetic storms. The data range used spans June 10, 1990 to June 10, 2021 (GMT/UTC); the Beaufort, NC tide gauge was installed in June 1990 and the WIS ONELINES time series extends through 2021. Note that the time series from the NOAA tide gauge were downloaded at year-long intervals, combined into a single CSV, and then a custom datetime was created ('dd-mmm-yyyy hh:mm:ss') prior to exporting as a txt file. I also had to manually delete NANs from the water level time series (January 1992).

<img align="left" width="500" height="300" src="FIGS/WIS.png"> 
<img align="right" width="450" height="600" src="FIGS/WIS_output.png"> 
<img align="left" width="500" height="600" src="FIGS/TidesCurrents.png">

We specify a berm elevation of 1.46 m NAVD88 and create two different storm lists for a beach slope of 0.03 and 0.002; the upper value corresponds to the average slope from the shoreline to the dune toe for Configuration X and the lower value Congifuration X. I could not get the vine copula function in R to compile, so I only tested the built-in elliptical copulas in Matlab (Gaussian and T-student). The T-student produced the highest $\tau$ values (Kendall's Corelation Coefficient) and is therefore what was used to create the synthetic storms. The function calls are as follows:

`[stStorms, stSimStorms] = multivariateSeaStorm("t", ...
"ST63266-time_series-061090_061021.onlns", ...
"CO-OPS_8656483_COMBINED_Reduced.txt", 0.03, 1.46, 20000, ...
true, "StormList_20k_NCB_Berm1pt46m_Slope0pt03_t-student.csv", ...
"/Users/KatherineAnardeWheels/PycharmProjects/CASCADE/notebooks/synthetic_storm_creation_NCB/data", ...
1990, 24)`

`[stStorms, stSimStorms] = multivariateSeaStorm("t", ...
"ST63266-time_series-061090_061021.onlns", ...
"CO-OPS_8656483_COMBINED_Reduced.txt", 0.002, 1.46, 20000, ...
true, "StormList_20k_NCB_Berm1pt46m_Slope0pt002_t-student.csv", ...
"/Users/KatherineAnardeWheels/PycharmProjects/CASCADE/notebooks/synthetic_storm_creation_NCB/data", ...
1990, 24)`

<img align="left" width="500" height="600" src="FIGS/NCB_TimeSeries_Corrected.png">  

Within `multivariateSeaStorm.m`, we first process the data by removing the 365-day (1 yr) running median. This differs from the 30-day running median used in Wahl et al. (2016), namely because we desired to maintain seasonal trends and only account for non-stationarity in wave and water level parameters due to inter-annual and decadal variability. The median of the last 3 years is then applied to the entire time series such that the new time series is representative of the current climate. A year-by-year tidal analysis is performed using t_Tide (Pawlowicz et al., 2002) to obtain the tidal amplitude $\eta_A$ and non-tidal residual $\eta_{NTR}$. Lastly, the representative highest elevation of the landward margin of runup (Rhigh, i.e. the total water level) is calculated as the sum of the maximum 2% exceedance of runup, following Stockdon et al. (2006), and the contemporaneous (corrected) water level elevation from the tide gauge. The representative lowest runup elevation (Rlow), below which any part of the barrier is considered continuously subaqueous during a storm, is taken as Rhigh – swash/2 (Figure 1). While wave direction is processed as part of the MSSM, it is not required as input for Barrier3D and is therefore not discussed herein.

### Figure 1. Timeseries corrected for non-stationarity due to inter-annual and decadal variability 

<img align="left" width="500" height="600" src="FIGS/0pt03_HsThreshold.png">
<img align="left" width="500" height="600" src="FIGS/0pt002_HsThreshold.png">

Storm events are then extracted from the corrected time series using the same metrics as Wahl et al. (2016), i.e., we derive a set of storm events by conditioning on Hs. Events are identified as periods of 8 or more consecutive hours with deep-water significant wave heights greater than 1.75 m (2.2 m) for the case of 0.03 (0.002) beach slope, or rather the minimum monthly averaged wave height for periods in which waters levels exceed the berm elevation (Figure 2). The remaining variables used to define the independent multivariate storm events for use in Barrier3D include $\eta_A$, $\eta_{NTR}$, $Tp$, and storm duration. We discard storms with concomitant values of surge that are negative and identify new storms when Hs drops below the threshold for 24 hours or more (cf. Li et al., 2014). This method results in 786 (410) independent multivariate sea-storm events for a 0.03 (0.002) beach slope.

### Figure 2. Wave height threshold used to define storms (i.e., the minimum monthly averaged wave height for periods when the TWL exceeded the berm elevation) for a beach slope of 0.03 (top) and 0.002 (bottom).

<img align="left" width="600" height="600" src="FIGS/0pt03_Histograms.png">
<img align="left" width="600" height="600" src="FIGS/0pt002_Histograms.png">

### Figure 3. Histograms of the empirical and synthetic storms for a beach slope of 0.03 (top) and 0.002 (bottom).

We use the approach of Wahl et al. (2016) for modeling the interdependencies between sea-storm variables through the use of elliptical copulas. The MvCAT toolbox (Sadegh et al., 2017) is first used to find marginal distributions that best fit each variable ($\eta_{NTR}$: Weibull; Hs and storm duration: Generalized Pareto; Tp: Generalized Extreme Value). The observed data are then transformed to the copula scale (the unit hypercube) by finding their rank and then rescaling the ranks by 1/(N+1) where N = the number of storm events. We then fit a T-Student copula to the transformed four-dimensional data set and generate 20,000 random samples (quadruplets). Lastly, we use the inverse of the fitted marginal CDFs to transform the simulated data from unit hypercube space back to the original scale of the data. As the tidal amplitude varies within a restricted range, we sample $\eta_A$ directly from its empirical CDF for each of the 20,000 synthetic storm events.

## Creating storm time series

Once the list of synthetic storms has been created, it can be used to generate annual storm time series (i.e., zero to many storms per year) for use in Barrier3D. The storm time series can be created using one of three functions, all of which are located in `tools.input_files`:
* `yearly_storms` - user specifies the mean and standard deviation for a normal distribution, which is then used to select a random number of storms to be pulled from the MSSM list each year.
* `frequency_storms` - user specifies the Rhigh (total water level) and frequency of a return period storm; the closest match to the Rhigh is found in the MSSM list and corresponding variables Rlow and duration are then simulated at the specified frequency.
* `shift_storm_intensity` - this function shifts the TWL distribution created in `yearly_storms` to the left or right to produce storms of different "intensity".

Below, we show example storm time series for a 100 year simulation using just the first method, which requires that we specify a mean and a standard deviation. For a beach slope of 0.03, the mean number of storms (from the list of historical storms) is 25.4 per year with a standard deviation of 5.3.

In [1]:
from pathlib import Path
from barrier3d.tools.input_files import yearly_storms

import os
import sys

os.getcwd()

sys.path

datadir = "./data"

yearly_storms(
    datadir=datadir,
    storm_list_name="StormList_20k_NCB_Berm1pt46m_Slope0pt03_t-student.csv",  # this is == "cascade_default_storm_list.csv"
    mean_yearly_storms=25.4, 
    SD_yearly_storms=5.3,  
    MHW=0.36,  # m NAVD88
    StormStart=2,
    BermEl=1.46,  # m NAVD88, just used for plotting
    model_years=100,
    bPlot=True,
    bSave=True,
    output_filename="StormSeries_100yrs_NCB_Berm1pt46m_Slope0pt03_01",
)

yearly_storms(
    datadir=datadir,
    storm_list_name="StormList_20k_NCB_Berm1pt46m_Slope0pt002_t-student.csv",  # this is == "cascade_default_storm_list.csv"
    mean_yearly_storms=13.2, 
    SD_yearly_storms=3.8,  
    MHW=0.36,  # m NAVD88
    StormStart=2,
    BermEl=1.46,  # m NAVD88, just used for plotting
    model_years=100,
    bPlot=True,
    bSave=True,
    output_filename="StormSeries_100yrs_NCB_Berm1pt46m_Slope0pt002_01",
)

ModuleNotFoundError: No module named 'pydantic'